In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm, model_selection

In [2]:
merged_path = "C:/Users/ozgur/Desktop/dev/Fantasy-Basketball-Player-Recomendation-System/ML/main_merged6.csv"
merged_df = pd.read_csv(merged_path, index_col=None, header=0)
merged_df.drop(['Pos', 'Date', 'STL_AVG', 'BLK Last Week',
       'USG_perc', 'DRtg', 'ORtg', 'AST_perc', 'DRB_perc', 'ORB_perc', 'Starter', 'MP Last Month',
       'BLK_perc', 'TOV_perc', 'STL_perc', 'eFG_perc', 'MP', 'FG', 'FGA', 'MP Last Week', 'PTS_AVG Last Week',
               'FG_perc', '3P', '3PA', '3P_perc', 'FT', 'FTA', 'BLK Last Month', 'TRB_AVG', 'PTS_AVG', '3PA_AVG',
       'FT_perc', 'TRB', 'STL', 'BLK', 'TOV', 'PF', 'W', 'BLK_AVG', 'FGA_AVG', 'FG_perc_AVG', 'FG_AVG Last Week',
       'FGA_AVG Last Week', 'FG_perc_AVG Last Week', 'FG_AVG Last Month', 'ORtg Last Month', 'TOV_AVG', 
       'FGA_AVG Last Month', 'FG_perc_AVG Last Month', 'AST Last Month', 'AST Last Week', 'AST_AVG', 'AST'], axis=1, inplace=True)

In [3]:
print(merged_df.columns)
print(merged_df.dtypes)
print(merged_df.shape)

Index(['Name', 'Team', 'Home', 'PTS', 'PG', 'SG', 'F', 'C',
       'PTS_AVG Last Month', 'MP_AVG', 'FG_AVG', 'Opp', 'ORtg_AVG',
       'eFG_perc_AVG', 'DRtg_AVG', 'USG_perc_AVG', '3P_AVG'],
      dtype='object')
Name                   object
Team                   object
Home                    int64
PTS                     int64
PG                      int64
SG                      int64
F                       int64
C                       int64
PTS_AVG Last Month    float64
MP_AVG                float64
FG_AVG                float64
Opp                    object
ORtg_AVG              float64
eFG_perc_AVG          float64
DRtg_AVG              float64
USG_perc_AVG          float64
3P_AVG                float64
dtype: object
(28911, 17)


In [4]:
merged_df['Team'] = pd.Categorical(merged_df['Team'])
merged_df['Opp'] = pd.Categorical(merged_df['Opp'])
dummies_team = pd.get_dummies(merged_df['Team'], prefix = 'Team')
dummies_opp = pd.get_dummies(merged_df['Opp'], prefix = 'Opp')
merged_df = pd.concat([merged_df, dummies_team], axis=1)
merged_df = pd.concat([merged_df, dummies_opp], axis=1)

In [ ]:
merged_df['Name'] = pd.Categorical(merged_df['Name'])
dummies_name = pd.get_dummies(merged_df['Name'], prefix = '')
merged_df = pd.concat([merged_df, dummies_name], axis=1)
merged_df

In [5]:
X_train = merged_df.loc[0:15500]
X_train = X_train.drop(['PTS', 'Name', 'Team', 'Opp'], 1)
X_norm = MinMaxScaler().fit_transform(X_train)
X_norm = pd.DataFrame(X_norm)
X_norm.columns = X_train.columns

In [6]:
y_train = merged_df.loc[1609:15000]
y_test = merged_df.loc[15000:15500]
x_train = X_norm.loc[1609:15000]
y_train = y_train[['PTS']]
x_test = X_norm.loc[15000:15500]
y_test = y_test[['PTS']]

In [ ]:
scorer = make_scorer(mean_squared_error, greater_is_better=False)
parameters = [{'kernel': ['rbf'], 'gamma': [0.01, 0.1, 0.2, 1e-3, 0,5], 'C': [1, 10, 100, 1000]},
        {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
        {'kernel': ['poly'], 'gamma': [0.01, 0.1, 0.2, 1e-3, 0,5], 'C': [1, 10, 100, 1000], 'degree' : [1.5, 2, 3] , 'epsilon' : [0.1, 0.5, 1, 10, 50]},
        {'kernel': ['sigmoid'], 'gamma': [0.01, 0.1, 0.2, 1e-3, 0,5], 'C': [1, 10, 100, 1000]}]

cv_svr = model_selection.GridSearchCV(svm.SVR(), parameters, cv = 3, n_jobs = -1, scoring = scorer)
cv_svr.fit(x_train, y_train)

In [ ]:
cv_svr.best_params_

In [25]:
svr1 = svm.SVR(kernel='poly', degree=3.5, epsilon=3, gamma=0.2, C=10)
svr1.fit(x_train, y_train)
y_pred = svr1.predict(x_test)
print(mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred))

c:\users\ozgur\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


32.67759139031497 0.6200682776497921


In [8]:
# 33.37901213478326 0.6119130868845086 degree:3
# 33.0951521420064 0.6152134346572027  degree:1.5
# 32.84124302335922 0.6181655533619073 degree:3 epsilon:1

32.84124302335922 0.6181655533619073
